Create and environment to use with this notebook:<br>
conda create -n ml3_final_project scipy scikit-image statsmodels scikit-learn pandas ipykernel tqdm keras pillow matplotlib pytorch==2.4.1 pytorch-cuda=12.4 torchinfo tensorflow=2.17 accelerate langchain pydub -c pytorch -c nvidia -c conda-forge

Install prerequisites uncomment and run once

In [1]:
# ! pip install parler-tts
# ! pip install spaces
# ! pip install accelerate
# ! pip install pydub
# ! pip install transformers
# ! pip install --upgrade protobuf
# ! pip install openai-whisper
# !pip install langchain_openai
# !pip install langchain_deepseek

Setup The LLM Part

In [2]:
# from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
conversation_history = []

In [4]:
def get_response(query):

    template = """Question: {question}

    Answer: """

    prompt = PromptTemplate.from_template(template)
    # llm =ChatDeepSeek(
    #         model="deepseek-chat",
    llm =ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            # other params...
        )
    llm_chain = prompt | llm
    question = f"You are a Doctor Michael's assistant interviewing a patient. Only ask one question at a time if needed.  Ask around 3 questions then tell the patient that you will relay the information to the doctor. Do not give answers as lists, and do not use any markup. Do not use apostrpohes to shorten words. Use word representations for numbers. Conversaion history is:{conversation_history} The patient now said: {query}"
    response = llm_chain.invoke(question).content
    interaction = {'patient':question, 'you':response}
    conversation_history.append(interaction)
    if len(conversation_history) > 7:
        del conversation_history[0]
    return response

Setup Whisper Speech To Text

In [5]:
from transformers import pipeline
import gradio as gr
import torch
import whisper

if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
# print("Using device", device)
model = whisper.load_model("base")
model.to(device)
# pipe = pipeline(model=model, device=device , return_timestamps=True)

def transcribe(audio):
    if audio == None:
      #  print('NO DATA')
       response = 'NO DATA GIVEN'
       return response
    try:
      text = model.transcribe(audio, language="en")["text"]
      if len(text.strip()) > 0:
        print(f'TRANSCRIBE {text}')
      return text
    except:
       response = 'ERROR TRANSCRIBING'
       return response

/home/andy/miniconda3/envs/ml3_final_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-19 16:02:16.951948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-19 16:02:16.971671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-19 16:02:16.977819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 16:02:16.993030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This 

Setup the Parler Text To Speech Model

In [6]:
from parler_tts import ParlerTTSStreamer
from parler_tts import ParlerTTSForConditionalGeneration

from transformers import AutoTokenizer, AutoFeatureExtractor, set_seed
import numpy as np
import spaces
import torch
from threading import Thread


device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch_dtype = torch.float16 if device != "cpu" else torch.float32

modelid_tiny = "parler-tts/parler-tts-tiny-v1"
print('LOADING PARLER TTS MODEL')
tts_model = ParlerTTSForConditionalGeneration.from_pretrained(
    modelid_tiny, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)

tokenizer = AutoTokenizer.from_pretrained(modelid_tiny)
feature_extractor = AutoFeatureExtractor.from_pretrained(modelid_tiny)

sampling_rate = tts_model.audio_encoder.config.sampling_rate
frame_rate = tts_model.audio_encoder.config.frame_rate
description = "Jenna speaks at an average pace with a calm delivery in a very confined sounding environment with clear audio quality."
description_tokens = tokenizer(description, return_tensors="pt").to(device)

print('READY')


Flash attention 2 is not installed


LOADING PARLER TTS MODEL


/home/andy/miniconda3/envs/ml3_final_project/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "r

READY


In [7]:
SAMPLE_RATE = feature_extractor.sampling_rate
SEED = 42
from pydub import AudioSegment
import io
import re

def numpy_to_mp3(audio_array, sampling_rate):
    # Normalize audio_array if it's floating-point
    if np.issubdtype(audio_array.dtype, np.floating):
        max_val = np.max(np.abs(audio_array))
        audio_array = (audio_array / max_val) * 32767 # Normalize to 16-bit range
        audio_array = audio_array.astype(np.int16)

    # Create an audio segment from the numpy array
    audio_segment = AudioSegment(
        audio_array.tobytes(),
        frame_rate=sampling_rate,
        sample_width=audio_array.dtype.itemsize,
        channels=1
    )

    # Export the audio segment to MP3 bytes - use a high bitrate to maximise quality
    mp3_io = io.BytesIO()
    audio_segment.export(mp3_io, format="mp3", bitrate="320k")

    # Get the MP3 bytes
    mp3_bytes = mp3_io.getvalue()
    mp3_io.close()

    return mp3_bytes

sampling_rate = tts_model.audio_encoder.config.sampling_rate
frame_rate = tts_model.audio_encoder.config.frame_rate

def split_text(text):
    text = text.replace('(','').replace(')','')
    phrases = re.split(r'(\.\s+|\n|\?\s+|\!\s+|\:\s+)', text)
    # print(phrases)
    reconstructed = [ ''.join(x) for x in zip(phrases[0::2], phrases[1::2])]
    reconstructed.append(phrases[-1])
    stripped = [x.strip().replace('.','') for x in reconstructed if len(x.strip()) > 1]
    # print(reconstructed)
    return stripped

In [8]:
@spaces.GPU
def read_response(answer):
    print(f'READING RESPONSE {answer}')

    play_steps_in_s = 3.0
    play_steps = int(frame_rate * play_steps_in_s)


    phrases = split_text(answer)
    
    for phrase in phrases:
        streamer = ParlerTTSStreamer(tts_model, device=device, play_steps=play_steps)
        prompt = tokenizer(phrase, return_tensors="pt").to(device)

        generation_kwargs = dict(
            input_ids=description_tokens.input_ids,
            prompt_input_ids=prompt.input_ids,
            streamer=streamer,
            do_sample=True,
            temperature=1.0,
            min_new_tokens=10,
        )

        set_seed(42)
        thread = Thread(target=tts_model.generate, kwargs=generation_kwargs)
        thread.start()

        for new_audio in streamer:
            print(f"Sample of length: {round(new_audio.shape[0] / sampling_rate, 2)} seconds")
            yield phrase, numpy_to_mp3(new_audio, sampling_rate=sampling_rate)

Launch the Web Interface

In [ ]:
with gr.Blocks() as block:
    gr.HTML(
        f"""
        <h1 style='text-align: center;'>Cutiepies Healthcare Assistant Chatbot</h1>
        <h3 style='text-align: center;'> Ask a question !</h3>
        """
    )
    with gr.Group():
        with gr.Row():
            audio_out = gr.Audio(label="Spoken Answer", streaming=True, autoplay=True)
            query = gr.Textbox(label="Query")
            answer = gr.Textbox(label="Answer")
            state = gr.State()
        with gr.Row():
            audio_in = gr.Audio(label="Speak your question", sources="microphone", streaming=False, type="filepath")

    audio_in.stop_recording(transcribe, inputs = audio_in, outputs = query)\
            .then(get_response, inputs = query, outputs = answer)\
            .then(fn=read_response, inputs=answer, outputs=[answer, audio_out])



block.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [10]:
# resp = read_response('Hello, My Name Is Andy')

# for item in resp:
#     print(item)

TRANSCRIBE  Hello, it's a doctor in right now.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


READING RESPONSE Hello! Doctor Michael is not available at the moment, but I am here to assist you. Can you please tell me what brings you in today?
Sample of length: 1.39 seconds
Sample of length: 2.41 seconds
Sample of length: 1.11 seconds
Sample of length: 2.64 seconds
TRANSCRIBE  Why is your voice so strange? Anyway, I've had a handicap for around five days now. It's just a handicap. No other symptoms.
READING RESPONSE I am sorry if my voice seems strange. I am here to help gather information for Doctor Michael. Can you describe what you mean by a handicap?
Sample of length: 2.1 seconds
Sample of length: 2.41 seconds
Sample of length: 0.64 seconds
Sample of length: 2.39 seconds
TRANSCRIBE  Sorry, I meant headache.
READING RESPONSE Thank you for clarifying. Can you tell me where exactly you feel the headache and if it is constant or comes and goes?
Sample of length: 1.85 seconds
Sample of length: 2.41 seconds
Sample of length: 1.97 seconds
TRANSCRIBE  I feel the headache in my head 